<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Torch_Notebooks/Fine_Tunning_Masked_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tunning pre-trained Masked language model

We will be using Huggingface pretrained T5 model for language translation task. The model is already pretained for on this task but we will fine tune it.

In [1]:
pip install -q pytorch_lightning datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizerFast,DistilBertForMaskedLM
from datasets import load_dataset


import dataset from kaggle

In [3]:
data = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
len(data["train"])

25000

In [6]:
# concatenate data
nn = []
for i in range(len(data["train"])):
  txt = (data["train"][i]["text"])
  nn.extend(txt.split(" "))

In [7]:
len(nn)

5844418

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [14]:
block_size = 64
batch_size = 4
x ,y =[],[]
for i in range(0,int(len(nn)*0.5),32):
  xb = nn[i:block_size+i]
  yb = nn[i:block_size+i]
  # ran = torch.randint(1,len(xb),(1,))
  # xb[ran] = tokenizer.mask_token
  x.append(xb)
  y.append(yb)


In [15]:
len(x)

91320

In [16]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = y
  def __len__(self):
    return len(self.x)

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]

In [17]:
mydata = MyDataset(x,y)

In [18]:
len(mydata)

91320

In [19]:
mydata[91319]

(['an',
  'ABC-TV',
  'celebration',
  'of',
  'show',
  'business',
  'nostalgia.<br',
  '/><br',
  '/>Nostalgia',
  'is',
  'what',
  '"Sunshine',
  'Boys"',
  'has',
  'going',
  'for',
  'it',
  'in',
  'spades,',
  'right',
  'from',
  'the',
  'start',
  'when',
  'a',
  'series',
  'of',
  '1920s',
  'film',
  'clips',
  'showcasing',
  'various',
  'entertainers',
  'from',
  'long',
  'ago',
  'flickers',
  'before',
  'us',
  'to',
  'the',
  'accompaniment',
  'of',
  'Cole',
  "Porter's",
  '"Be',
  'A',
  'Clown".',
  'Then',
  "there's",
  'the',
  "film's",
  'present-day',
  'setting',
  'in',
  'Manhattan,',
  'where',
  'flared',
  'trousers',
  'and',
  'wide',
  'polyester',
  'ties',
  'abound.'],
 ['an',
  'ABC-TV',
  'celebration',
  'of',
  'show',
  'business',
  'nostalgia.<br',
  '/><br',
  '/>Nostalgia',
  'is',
  'what',
  '"Sunshine',
  'Boys"',
  'has',
  'going',
  'for',
  'it',
  'in',
  'spades,',
  'right',
  'from',
  'the',
  'start',
  'when',
  '

In [22]:
for x in mydata:
  inputs = tokenizer(x[0],is_split_into_words=True,padding=True,truncation=True,return_tensors="pt")
  labels = tokenizer(x[1],is_split_into_words=True,padding=True,truncation=True,return_tensors="pt")
  rand = torch.randint(1,len(inputs["input_ids"][0]),(2,))
  inputs["input_ids"][0][rand] = tokenizer.mask_token_id
  labels = labels["input_ids"]
  inputs["labels"] = labels
  print(inputs)
  print(tokenizer.decode(inputs["input_ids"][0]))
  print(tokenizer.decode(labels[0]))
  break

{'input_ids': tensor([[  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
          2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
          2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
          2657,  2008,  2012,  2034,  2009,  2001,  8243,  2011,  1057,  1012,
          1055,  1012,  8205,  2065,  2009,  2412,  2699,  2000,  4607,  2023,
          2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,  1000,
          6801,  1000,  1045,   103,  2018,  2000,  2156,  2023,  2005,  2870,
          1012,  1026,  7987,  1013,  1028,  1026,  7987,   103,  1028,  1996,
          5436,  2003,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tenso

In [23]:
def collate_fn(batch):
  inputs = [x[0] for x in batch]
  targets = [x[1] for x in batch]
  inputs = tokenizer(inputs,is_split_into_words=True, padding=True,truncation=True,return_tensors="pt")
  yb = tokenizer(targets,is_split_into_words=True,padding=True,truncation=True,return_tensors="pt")
  rand = torch.randint(1,len(inputs["input_ids"][0]),(2,))
  inputs["input_ids"][0][rand] = tokenizer.mask_token_id
  yb = yb["input_ids"]
  yb[yb==tokenizer.pad_token_id] = -100
  inputs["labels"] = yb
  return inputs

In [24]:
train,val = torch.utils.data.random_split(mydata,[90000,1320])

In [25]:
train_df = torch.utils.data.DataLoader(train,batch_size=32,shuffle=True,collate_fn=collate_fn,drop_last=True)
val_df = torch.utils.data.DataLoader(val,batch_size=32,collate_fn=collate_fn,drop_last=True)

In [28]:
for i,x in enumerate(train_df):
  if i == 1:
    break
  print(x["input_ids"].shape)
  print(x["attention_mask"].shape)
  print(x["labels"].shape)
  print(x)


torch.Size([32, 117])
torch.Size([32, 117])
torch.Size([32, 117])
{'input_ids': tensor([[  101,  2028, 20587,  ...,     0,     0,     0],
        [  101,  2156,  2054,  ...,     0,     0,     0],
        [  101,  6719,  2038,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  6841,  ...,     0,     0,     0],
        [  101,  2242,  2182,  ...,     0,     0,     0],
        [  101,  2436,  1012,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  2028, 20587,  ...,  -100,  -100,  -100],
        [  101,  2156,  2054,  ...,  -100,  -100,  -100],
        [  101,  6719,  2038,  ...,  -100,  -100,  -100],
        ...,
        [  101,  1996,  6841,  ...,  -100,  -100,  -100],
        [  101,  2242,  2182,  ...,  -100,  -100,  -100],
       

Pass is_split_into_words=True as out sentences are already split into words.

Initialise the model with number of labels we have in our database.

In [29]:
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Create a lightning module and set low learning rate of 0.0001. We will use the built-in loss function of model.

In [30]:
class MyModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = model
    self.loss = torch.nn.CrossEntropyLoss()

  def forward(self,input_ids,attention_mask):
    return self.model(input_ids=input_ids,attention_mask=attention_mask)

  def training_step(self,batch,batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    outputs = self.model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
    # B,T,C = outputs.logits.shape
    # loss = self.loss(outputs.logits.view(B*T,C),labels.view(B*T))
    loss = outputs.loss
    self.log("train_loss",loss,prog_bar=True,logger=True)
    return loss

  def validation_step(self,batch,batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    outputs = self.model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
    # B,T,C = outputs.logits.shape
    # loss = self.loss(outputs.logits.view(B*T,C),labels.view(B*T))
    loss = outputs.loss
    self.log("val_loss",loss,prog_bar=True,logger=True)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=0.0001)

In [31]:
pl_model = MyModel()

In [32]:
trainer = pl.Trainer(max_epochs=1)
trainer.fit(pl_model,train_df,val_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                  | Params | Mode 
--------------------------------------------------------
0 | model | DistilBertForMaskedLM | 67.0 M | eval 
1 | loss  | CrossEntropyLoss      | 0      | train
--------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
267.942   Total estimated model params size (MB)
1         Modules in train mode
98        Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
for x in mydata:
  print(x)
  break

(['I', 'rented', 'I', 'AM', 'CURIOUS-YELLOW', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first', 'released', 'in', '1967.', 'I', 'also', 'heard', 'that', 'at', 'first', 'it', 'was', 'seized', 'by', 'U.S.', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', 'this', 'country,', 'therefore', 'being', 'a', 'fan', 'of', 'films', 'considered', '"controversial"', 'I', 'really', 'had', 'to', 'see', 'this', 'for', 'myself.<br', '/><br', '/>The', 'plot', 'is'], ['I', 'rented', 'I', 'AM', 'CURIOUS-YELLOW', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first', 'released', 'in', '1967.', 'I', 'also', 'heard', 'that', 'at', 'first', 'it', 'was', 'seized', 'by', 'U.S.', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', 'this', 'country,', 'therefore', 'being', 'a', 'fan', 'of', 'films', 'considered', '"controversial"', 'I', 're

In [135]:
tt = "As hard to believe as it might seem, this series is also 'fresh'. It is truly original - it hasn't been done before. Midway through the first [MASK], something else of significance occurs, and somehow the makers of this drama managed to convince the people who make the trailers not to include it - or even allude to it."
inputs = tokenizer(tt,is_split_into_words=False,return_tensors="pt")
with torch.no_grad():
  out = pl_model(inputs["input_ids"],inputs["attention_mask"])
masked_probs = torch.nn.functional.softmax(out.logits[0][inputs["input_ids"][0] == tokenizer.mask_token_id], dim=-1)
top3 = torch.topk(masked_probs,k=5,dim=-1)
print(top3.indices)
for token in top3.indices:
  print(tokenizer.decode(token.flatten().tolist()))



tensor([[2143, 2051, 3496, 2466, 3185]])
film time scene story movie


In [134]:
tt = "As hard to believe as it might seem, this series is also 'fresh'. It is truly original - it hasn't been done before. Midway through the first [MASK], something else of significance occurs, and somehow the makers of this drama managed to convince the people who make the trailers not to include it - or even allude to it."
inputs = tokenizer(tt,is_split_into_words=False,return_tensors="pt")
with torch.no_grad():
  out = pl_model(inputs["input_ids"],inputs["attention_mask"])
pred = torch.argmax(out.logits,dim=-1)
print(tokenizer.decode(pred[0]))

[CLS] as hard to believe as it might seem, this series is also ' fresh '. it is truly original - it hasn ' t been done before. midway through the first film, something else of significance occurs, and somehow the makers of this drama managed to convince the people who make the trailers not to include it - or even allude to it. [SEP]
